# 加入出发地当天天气信息

In [ ]:
import pandas as pd

In [ ]:
# 创建出发地天气id字典
airport = pd.read_csv('./dataset/temp_airport.csv', encoding= 'gbk')
# 读取airport.csv文件为字典
weather_dict = {}
for i in range(len(airport)):
    weather_dict[airport['机场编码'][i]] = airport['网站ID'][i]
print(weather_dict)

# 进入网站爬取数据


In [ ]:
#将id变为str
from Write import write
for city in weather_dict:
    id = str(weather_dict[city])
    write(city, id)
    print(city, '(', id, ')已写入csv文件')

# 将气象数据合并至flight_reduced_calculated.csv中并作为数据集

In [ ]:
# 读取flight_reduced_unique_calculated.csv文件
dataSet = pd.read_csv('./dataset/flight_reduced_calculated.csv', encoding='utf-8')

# 创建出发地天气id字典
airport = pd.read_csv('./dataset/airport.csv', encoding= 'gbk')
# 读取airport.csv文件为列表
airport_list = []
for i in range(len(airport)):
    airport_list.append(airport['机场编码'][i])

# 删除无气象数据的行
dataSet = dataSet[dataSet['出发机场'].isin(airport_list)]
dataSet = dataSet[dataSet['到达机场'].isin(airport_list)]

# 读取各机场的天气csv文件
airport_weather = {}
for i in range(len(airport_list)):
    airport_weather[airport_list[i]] = pd.read_csv('./dataset/weather/' + airport_list[i] + '.csv', encoding='utf-8')

drop_list = []

# 添加天气数据到原始数据集
dataSet['平均温度'] = float(0)
dataSet['最高温度'] = float(0)
dataSet['最低温度'] = float(0)
dataSet['降水量'] = float(0)
dataSet['气压'] = float(0)
dataSet['风向'] = float(0)
dataSet['风速'] = float(0)
for i in range(len(dataSet)):
    # 通过机场编码和日期找到对应的天气数据
    airport_code = dataSet['出发机场'].values[i]
    date = dataSet['计划出发日期'].values[i]
    # 将日期变为字符串
    date = str(date).split(' ')[0].split('-')[2] + '/' + str(date).split(' ')[0].split('-')[1] + '/' + str(date).split(' ')[0].split('-')[0]
    # 找到对应日期的天气数据
    for j in range(len(airport_weather[airport_code])):
        if airport_weather[airport_code]['Time'][j] == date:
            dataSet['平均温度'].values[i] = airport_weather[airport_code]['Ave_t'].values[j]
            dataSet['最高温度'].values[i] = airport_weather[airport_code]['Max_t'].values[j]
            dataSet['最低温度'].values[i] = airport_weather[airport_code]['Min_t'].values[j]
            dataSet['降水量'].values[i] = airport_weather[airport_code]['Prec'].values[j]
            dataSet['气压'].values[i] = airport_weather[airport_code]['SLpress'].values[j]
            dataSet['风向'].values[i] = airport_weather[airport_code]['Winddir'].values[j]
            dataSet['风速'].values[i] = airport_weather[airport_code]['Windsp'].values[j]
            # print(airport_weather[airport_code]['Ave_t'].values[j])
            # print(dataSet['平均温度'].values[i])
            break
    if i%1000 == 0:
        print(i/len(dataSet)*100, '%')

# 保存数据集
dataSet.to_csv('./dataset/flight_final.csv', encoding='utf-8',index = False)


# 删除没有气象数据的行

In [ ]:
# 读取flight_final.csv文件
dataSet = pd.read_csv('./dataset/flight_final.csv', encoding='utf-8')

# 删除平均温度，最高温度，最低温度，降水量，气压，风向，风速为0的行
dataSet = dataSet[dataSet['平均温度'] != 0]

# 保存数据集
dataSet.to_csv('./dataset/flight_final_no_zero.csv', encoding='utf-8',index = False)